In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-o704g6k0
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-o704g6k0
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=7a010b6ab931bb6da774b1b3ef6a7fd6044236d1079c28af3de5d46f5d44e529
  Stored in directory: /tmp/pip-ephem-wheel-cache-rl9gkp2c/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from gensim.models import KeyedVectors

wv = KeyedVectors.load('/content/drive/MyDrive/Colab Notebooks/word_vectors/word2vec_v8.bin', mmap='r')

In [ ]:
import pandas as pd
import numpy as np
import re

import tensorflow as tf
import matplotlib.pyplot as plt

from nltk import word_tokenize

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/CV Parser - Labeled Data_v6.csv', encoding='latin')
data.tail()

,cv,line,tag
35854,533.pdf,WORRKING SPRITE,O
35855,533.pdf,"Id like eager to learn, good team working an...",O
35856,533.pdf,solving problem.,O
35857,533.pdf,Strongly skill Analyst and Logical thinking.,O
35858,533.pdf,Funny and tolerate high work pressure.,O


In [ ]:
def decontracted(phrase):
    # specific
    phrase = re.sub(":|-\s+|,|–\s+|-|\||\/", " ", phrase)
    phrase = re.sub(r"â€|™|“|”|;|ü|\xad|\xa0|\u200b|·|∙|�|●|�|§|•|!|▪|©|\?|\]|\[|\)|\(", "", phrase)
    phrase = re.sub(r"[^\w\s]", " ", phrase)
    
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    
    phrase = re.sub(r"\bhcmc\b", "ho chi minh city ", phrase)
    phrase = re.sub(r"\bhcm\b", "ho chi minh ", phrase)
    phrase = re.sub(r"\bbs\b ", "bachelor of ", phrase)
    phrase = re.sub(r"r’s", "r of", phrase)
    phrase = re.sub(r"\’s", " of", phrase)
    
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " of", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\bim\b", "i am ", phrase)
    phrase = re.sub(r"\bi’m\b", "i am ", phrase)
    
    return phrase

def sent2idx(x):
    xidx = []
    for w in word_tokenize(x):
        if w in wv.key_to_index:
            xidx.append(wv.key_to_index[w])
    return np.array(xidx)

In [ ]:
line = []
for i in data['line']:
    regex = decontracted(str(i).lower())
    line.append(regex.strip())

s2v = []
for i in line:
    s2v.append(sent2idx(i))

In [ ]:
for i, x in enumerate(s2v):
    if len(x) < 64:
        s2v[i] = np.hstack((x, np.zeros(64-len(x))))
X = np.array(s2v)
X

array([[ 1363.,   575.,  1155., ...,     0.,     0.,     0.],
       [  253.,   271.,     0., ...,     0.,     0.,     0.],
       [  501.,  3516.,   234., ...,     0.,     0.,     0.],
       ...,
       [  327.,   268.,     0., ...,     0.,     0.,     0.],
       [ 2994.,    80.,   476., ...,     0.,     0.,     0.],
       [ 2401.,     0., 10038., ...,     0.,     0.,     0.]])

In [ ]:
from keras.models import load_model

from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy

model = load_model('/content/drive/MyDrive/Colab Notebooks/final/model/bilstm_crf_v4.h5', custom_objects={'CRF': CRF, 
                                                                                                 'crf_loss': crf_loss , 
                                                                                                 'crf_viterbi_accuracy': crf_viterbi_accuracy
                                                                                                 })

In [ ]:
# # tag for model version 2
# idx2tag = {0: 'O', 1: 'I-W', 2: 'I-S', 3: 'B-E', 4: 'B-I', 5: 'I-I', 6: 'B-S', 7: 'B-SU', 8: 'I-E', 9: 'B-W', 10: 'I-SU'}
# tag2idx = {'O': 0, 'I-W': 1, 'I-S': 2, 'B-E': 3, 'B-I': 4, 'I-I': 5, 'B-S': 6, 'B-SU': 7, 'I-E': 8, 'B-W': 9, 'I-SU': 10}

# tag for model version 4
tag2idx = {'I-W': 0, 'nan': 1, 'B-SU': 2, 'I-S': 3, 'B-I': 4, 'I_I': 5, 'B-S': 6, 'I-I': 7, 'I-E': 8, 'I-SU': 9, 'B-E': 10, 'B-Su': 11, 'O': 12, 'B-W': 13} 
idx2tag = {0: 'I-W', 1: 'nan', 2: 'B-SU', 3: 'I-S', 4: 'B-I', 5: 'I_I', 6: 'B-S', 7: 'I-I', 8: 'I-E', 9: 'I-SU', 10: 'B-E', 11: 'B-Su', 12: 'O', 13: 'B-W'}

n_tags = len(tag2idx)
print(tag2idx, '\nn_tag:' ,n_tags)

{'I-W': 0, 'nan': 1, 'B-SU': 2, 'I-S': 3, 'B-I': 4, 'I_I': 5, 'B-S': 6, 'I-I': 7, 'I-E': 8, 'I-SU': 9, 'B-E': 10, 'B-Su': 11, 'O': 12, 'B-W': 13} 
n_tag: 14


In [ ]:
lst = ['a', 'b', 'c']
lst.append('')
lst

['a', 'b', 'c', '']

In [ ]:
pre_model = []
for i in X:
    p = model.predict(np.array([i]))
    p = np.argmax(p)
    pre_model.append(idx2tag[p])
    print('\r line: ', len(pre_model), end='')
    #print("{:90}: {:5}".format(w, idx2tag[p]))

 line:  35859

In [ ]:
prediction = pre_model

In [ ]:

data['pre_tag_with_model'] = prediction

In [ ]:
data.to_csv('data_evaluate_V2.csv')

In [ ]:
# import os

# files = []
# path = '/content/drive/MyDrive/Colab Notebooks/txt_cv_1-533/'
# for f in os.listdir(path):
#     if f.endswith('.txt'):
#         files.append(f)

In [ ]:
# import re

# test_line = []
# pre_model = []

# for fi in files:

#     nfile = '/content/drive/MyDrive/Colab Notebooks/txt_cv_1-533/' + fi
#     with open(nfile, 'r', encoding = 'utf-8') as f:
#         raw = f.readlines()
#         for i in raw:
#             clean = decontracted(str(i).lower())
#             test_line.append(clean.strip())
#     n_line = len(raw)
#     test_data = []
#     for i in test_line:
#         test_data.append(sent2idx(i))

#     for i, x in enumerate(test_data):
#         if len(x) < 64:
#             test_data[i] = np.hstack((x, np.zeros(64-len(x))))


    #p = model.predict(np.array([test_data[i]]))
    #p = np.argmax(p, axis=-1)
    # Visualization
    # print("{:90}||{:5}".format("sentences", "pre tag"))
    # print(100 * "=")
#     for w, i in zip(test_line, test_data):
#         p = model.predict(np.array([i]))
#         p = np.argmax(p)
#         pre_model.append(idx2tag[p])
#         print('\r line: ', len(pre_model), end='')
#         #print("{:90}: {:5}".format(w, idx2tag[p]))
# data['pre_tag_with_model'] = pre_model

 line:  59513

KeyboardInterrupt: ignored